<a href="https://colab.research.google.com/github/DmitriySechkin/ds-learning-sb/blob/main/NLP2_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NLP2_1
https://www.hackerrank.com/challenges/detect-the-email-addresses/problem?isFullScreen=true

NLP2_2 https://www.hackerrank.com/challenges/detect-the-domain-name/problem?isFullScreen=true

NLP2_3 (дз1): Реализовать stemming, lemmatization & BoW на следующем датасете: https://cloud.mail.ru/public/Z4L3/vB8GcgTtK (Russian Toxic-abuse comments)

NLP2_4 Реализовать классификатор токсичных комментариев tfidf на базе датасета
https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments  

Дубликат файла: https://cloud.mail.ru/public/Z4L3/vB8GcgTtK



In [18]:
# NLP2_1

import re

n = int(input())
result = []
patt = r'[\w.]+@[\w.]+\w'

for i in range(n):
    string = input()
    res = re.findall(patt, string)

    if res:
      result.extend(res)

mails = list(set(result))
mails.sort()

print(';'.join(mails))

1
Mail us at hackers@hackerrank.com to chat more. Or you can write to us at interviewstreet@hackerrank.com!
hackers@hackerrank.com;interviewstreet@hackerrank.com


In [19]:
# NLP2_2

import re

n = int(input())
result = []
patt = r'https?://(?:ww[w2]\.)?([^/"\']+?\.\w+)[/?"]'

for i in range(n):
    string = input()
    res = re.findall(patt, string)

    if res:
      result.extend(res)

domens = list(set(result))
domens.sort()

print(';'.join(domens))


1
<div class="reflist" style="list-style-type: decimal;"> <ol class="references"> <li id="cite_note-1"><span class="mw-cite-backlink"><b>^ ["Train (noun)"](http://www.askoxford.com/concise_oed/train?view=uk). <i>(definition – Compact OED)</i>. Oxford University Press<span class="reference-accessdate">. Retrieved 2008-03-18</span>.</span><span title="ctx_ver=Z39.88-2004&rfr_id=info%3Asid%2Fen.wikipedia.org%3ATrain&rft.atitle=Train+%28noun%29&rft.genre=article&rft_id=http%3A%2F%2Fwww.askoxford.com%2Fconcise_oed%2Ftrain%3Fview%3Duk&rft.jtitle=%28definition+%E2%80%93+Compact+OED%29&rft.pub=Oxford+University+Press&rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Ajournal" class="Z3988"><span style="display:none;"> </span></span></span></li> <li id="cite_note-2"><span class="mw-cite-backlink"><b>^</b></span> <span class="reference-text"><span class="citation book">Atchison, Topeka and Santa Fe Railway (1948). <i>Rules: Operating Department</i>. p. 7.</span><span title="ctx_ver=Z39.88-2004&rfr_id=in

In [ ]:
# NLP2_3


In [1]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 731.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=ac8848c24d330bb1ace8a126d48a034642c2ecea5d1b185631441f97f688243e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
import pandas as pd
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem.snowball import SnowballStemmer

import itertools
import string

import numpy as np
from numpy import dot
from numpy.linalg import norm

from nltk.tokenize import WordPunctTokenizer

import pymorphy2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('drive/MyDrive/labeled.csv', delimiter=',')

X = data.comment.values.tolist()
y = data.toxic.values.tolist()

In [72]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [5]:
def is_target_token(token, special_char):
  return token.isalpha() and \
          token not in special_char and \
            token not in stopwords.words('russian')

def process_data(data):
  special_char=[",",":"," ",";",".","?", "+", '"?']

  tokenizer = WordPunctTokenizer()
  stemmer = SnowballStemmer('russian')
  morph = pymorphy2.MorphAnalyzer()

  data_tok = []

  for i in range(len(data)):
    data_tok.append(tokenizer.tokenize(data[i].lower()))

  preprocessed_data = []

  for sent in data_tok:
    n_sent = [stemmer.stem(morph.parse(t)[0].normal_form) for t in sent if is_target_token(t, special_char)]
    preprocessed_data.append(n_sent)

  assert(len(preprocessed_data) == len(data_tok))

  return preprocessed_data

In [6]:
preprocessed_data = process_data(X)

unique_vocab = list(set(list(itertools.chain.from_iterable(preprocessed_data))))
unique_vocab.remove('')

def vectorize(tokens, vocab):
    vector=[]
    for w in vocab:
        vector.append(tokens.count(w))
    return vector

bow = []

for i in preprocessed_data:
  bow.append(vectorize(i, unique_vocab))

assert(len(X) == len(bow))

In [7]:
# NLP2_4

!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.8 MB/s eta 0:00:00


In [12]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import optuna
import logging

from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [13]:
def vectorize(preprocessed_data, count_vect, tfidf_vect):
  preprocessed_data_str = [' '.join(i) for i in preprocessed_data]

  word_count = count_vect.fit_transform(preprocessed_data_str)

  tfidf_vect.fit(word_count)
  tf_idf_vector = tfidf_vect.transform(word_count).toarray()

  assert(len(tf_idf_vector) == len(preprocessed_data))

  return tf_idf_vector

In [37]:
count_v = CountVectorizer()
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tf_idf_vector = vectorize(preprocessed_data, count_v, tfidf_transformer)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(tf_idf_vector, y, test_size=0.2)

In [16]:
%%time
# optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):

    param = {
        "n_estimators" : trial.suggest_int("n_estimators", 0, 700, step=100),
        "max_depth": trial.suggest_int("max_depth", 3, 15, step=3),
        "learning_rate": trial.suggest_float("learning_rate", 0.1, 0.6, step=0.1),
        "verbosity": 0,
        "objective": "binary:logistic",
        "booster": "gbtree"
    }

    le = LabelEncoder()
    y_boost_encoded = le.fit_transform(y_train)

    bst = XGBClassifier(**param, gpu_id=0).fit(X_train, y_boost_encoded)
    y_pred_encoded = bst.predict(X_test)

    y_pred = le.inverse_transform(y_pred_encoded)

    accuracy = accuracy_score(y_pred, y_test)
    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)



[I 2024-03-24 21:48:11,432] A new study created in memory with name: no-name-90e80e7d-24f4-494d-98f8-cb59383dc279
[I 2024-03-24 21:49:07,664] Trial 0 finished with value: 0.8449531737773153 and parameters: {'n_estimators': 300, 'max_depth': 12, 'learning_rate': 0.5}. Best is trial 0 with value: 0.8449531737773153.
[I 2024-03-24 21:50:32,848] Trial 1 finished with value: 0.8352410683315991 and parameters: {'n_estimators': 700, 'max_depth': 15, 'learning_rate': 0.4}. Best is trial 0 with value: 0.8449531737773153.
[I 2024-03-24 21:51:15,435] Trial 2 finished with value: 0.8057578910856746 and parameters: {'n_estimators': 100, 'max_depth': 15, 'learning_rate': 0.1}. Best is trial 0 with value: 0.8449531737773153.
[I 2024-03-24 21:52:26,012] Trial 3 finished with value: 0.8414845646895595 and parameters: {'n_estimators': 500, 'max_depth': 15, 'learning_rate': 0.30000000000000004}. Best is trial 0 with value: 0.8449531737773153.
[I 2024-03-24 21:53:13,919] Trial 4 finished with value: 0.820

CPU times: user 14min 55s, sys: 9.55 s, total: 15min 5s
Wall time: 10min 51s


In [17]:
acc_xgboost_optuna = study.best_trial.value
acc_xgboost_optuna

0.8494623655913979

In [18]:
best_params = study.best_trial.params

In [19]:
le = LabelEncoder()
y_boost_encoded = le.fit_transform(y_train)

bst_model = XGBClassifier(**best_params, gpu_id=0).fit(X_train, y_boost_encoded)

In [69]:
def predict_toxic(value, model, count_vectorizer, tfidf_transformer):
    processed_value = process_data([value])

    vector = count_vectorizer.transform([' '.join(tokens) for tokens in processed_value])
    tfidf_vector = tfidf_transformer.transform(vector).toarray()

    prediction = model.predict(tfidf_vector)

    return 'toxic' if prediction[0] > 0.5 else "not toxic"

In [70]:
predict_toxic('кругом одни идиоты', bst_model, count_v, tfidf_transformer)

'toxic'

In [71]:
predict_toxic('кругом приятные люди', bst_model, count_v, tfidf_transformer)

'not toxic'